In [1]:
from qiskit import IBMQ

In [ ]:
IBMQ.enable_account(token=...)

In [ ]:
import numpy as np
import networkx as nx
from operator import itemgetter
import import_ipynb
import QLS_for_GB2 as qls

In [4]:
def compute_mod(G,k): #returns modularity for given consideration of node attributes
    #--> alternative approaches to yield maximum modularity: brute force search in space
    #of power set with detected communities (by recursive_QLS) as its elements
    for i in G.nodes:
        sum=0
        for j in range(0,k):
            sum=sum+pow(2,j)*G.nodes[i]["Spin{}".format(j)] #binary encoding of community by node attributes
        G.nodes[i]["colour"]=sum
    
    groups = set(nx.get_node_attributes(G,'colour').values())
    communities=list()
    
    for l in groups: #create propper community form for nx-function
        group=set()
        for i in G.nodes:
            if G.nodes[i]["colour"]==l:
                group.add(i)
        communities.append(group)
    
    result=nx.algorithms.community.quality.modularity(G, communities)
    return result, communities

In [5]:
def recursive_QLS(G,k, iter=0):
    if len(G)==0:
        return 0 #stopping criteria if there are no further nodes
    lis=list(G.nodes())
    n=len(lis)
    subgraph2graph=dict((x,lis[x]) for x in range (n))
    graph2subgraph=dict((lis[x],x) for x in range (n))
    
    result=qls.QLS(G, method='QAOA', backend='melbourne') #call QLS for partition into two communities
    solution=result[2]
    iter_time=(result[0])
    global total_time
    total_time+=iter_time
    
    if -1 in solution:
        solution = [0 if x == -1 else 1 if x == 1 else 'Error' for x in solution]
        
    
    for i,j in zip(G.nodes, range(0,len(G))): #add node attributes according to QLS solution
        G.nodes[i]["Spin{}".format(iter)]=solution[j]
        
    #create subgraphs for further recursions    
    subnodes1 = (node
                    for node, data
                    in G.nodes(data=True)
                    if data.get("Spin{}".format(iter)) == 1)
        
    subnodes2 = (node
                    for node, data
                    in G.nodes(data=True)
                    if data.get("Spin{}".format(iter)) == 0)
        
    G1 = G.subgraph(subnodes1)
    G2 = G.subgraph(subnodes2)
    
    
    iter+=1
    if iter>=k:
        return 0
    
    recursive_QLS(G1,k,iter)
    recursive_QLS(G2,k,iter)

In [ ]:
def GB2(G): #overall function specifically for Zachary Karate Club Graph (generalization stated in code/paper)
    k=3 #for this special case: Zachary Karate Club has 34 nodes
    #--> number of communities c=int(sqrt(34))+1, --> k=int(log2(c))+1=3
    
    total_time=0
    
    recursive_QLS(G,k)
    mod=compute_mod(G,2) #found to be highest modularity when considering 4 communities
    
    return (mod[0],total_time)